In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer
import xgboost as xgb
import lightgbm as lgb
import sklearn # For version check

In [2]:
train_df = pd.read_csv(r'C:\Users\kumara\Downloads\1a513221-3-dataset\dataset\train.csv')
test_df = pd.read_csv(r"C:\Users\kumara\Downloads\1a513221-3-dataset\dataset\test.csv")
sample_submission_df = pd.read_csv(r"C:\Users\kumara\Downloads\1a513221-3-dataset\dataset\sample_submission.csv")

In [3]:
train_df

,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
0,0,7.817315,576.179270,41.24308670850264,32.135501,4.0,0.803199,37.403527,1.963787,13.691147,62.494044,12.82491203459621,1018.8665053152533,A1,NaN,NaN,0.562096
1,1,24.785727,240.003973,1.3596482765960705,19.977460,8.0,0.479456,21.843315,0.241473,27.545096,43.851238,12.012043660984917,1025.6238537572883,D4,E00,dual-axis,0.396447
2,2,46.652695,687.612799,91.26536837560256,1.496401,4.0,0.822398,48.222882,4.191800,43.363708,NaN,1.814399755560454,1010.9226539809573,C3,E00,NaN,0.573776
3,3,53.339567,735.141179,96.19095521176159,18.491582,3.0,0.837529,46.295748,0.960567,57.720436,67.361473,8.736258932034128,1021.8466633134253,A1,NaN,dual-axis,0.629009
4,4,5.575374,12.241203,27.495073003585226,30.722697,6.0,0.551833,0.000000,0.898062,6.786263,3.632000,0.52268384077164,1008.5559577591927,B2,E00,fixed,0.341874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,16.868428,NaN,93.53031757838667,14.393967,3.0,0.738911,12.147711,3.005355,26.206810,1.733013,12.594122273332914,1018.3744670739436,B2,E02,tracking,0.664907
19996,19996,53.415061,296.970303,93.98571391279083,25.997012,2.0,0.513061,0.000000,0.532119,65.000000,64.558667,0.9769909288128159,1016.081102065643,D4,E00,fixed,0.354070
19997,19997,2.442727,660.328019,37.9689180401391,32.818396,9.0,0.548602,13.047950,4.075498,11.584869,57.730134,4.750937249871706,1009.6844614602336,D4,NaN,tracking,0.419734
19998,19998,NaN,632.760700,43.01470184078199,19.063517,4.0,NaN,0.000000,1.068906,21.149351,78.123689,11.304158443374758,1006.6738746072241,A1,E00,tracking,0.661963


In [4]:
train_df.shape

(20000, 17)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  20000 non-null  int64  
 1   temperature         18999 non-null  float64
 2   irradiance          19013 non-null  float64
 3   humidity            20000 non-null  object 
 4   panel_age           18989 non-null  float64
 5   maintenance_count   18973 non-null  float64
 6   soiling_ratio       18990 non-null  float64
 7   voltage             19007 non-null  float64
 8   current             19023 non-null  float64
 9   module_temperature  19022 non-null  float64
 10  cloud_coverage      18990 non-null  float64
 11  wind_speed          20000 non-null  object 
 12  pressure            20000 non-null  object 
 13  string_id           20000 non-null  object 
 14  error_code          14088 non-null  object 
 15  installation_type   14972 non-null  object 
 16  effi

In [6]:
train_df.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,20000.0,NaN,NaN,NaN,9999.5,5773.647028,0.0,4999.75,9999.5,14999.25,19999.0
temperature,18999.0,NaN,NaN,NaN,25.077241,12.513129,0.0,16.853522,24.720345,32.848917,147.394168
irradiance,19013.0,NaN,NaN,NaN,501.273896,250.92659,-597.278646,332.227277,499.65473,668.416734,1537.810349
humidity,20000,19876,unknown,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
panel_age,18989.0,NaN,NaN,NaN,17.509758,10.097557,0.001264,8.777905,17.497731,26.340761,34.998379
maintenance_count,18973.0,NaN,NaN,NaN,4.01207,2.002268,0.0,3.0,4.0,5.0,15.0
soiling_ratio,18990.0,NaN,NaN,NaN,0.698879,0.172244,0.400149,0.550654,0.697663,0.847838,0.999949
voltage,19007.0,NaN,NaN,NaN,16.242251,17.889031,0.0,0.0,12.350138,26.557322,494.279016
current,19023.0,NaN,NaN,NaN,1.713396,1.152953,0.000054,0.772311,1.558413,2.474744,7.315597
module_temperature,19022.0,NaN,NaN,NaN,29.923807,12.125405,0.0,21.522124,29.857669,38.094943,65.0


In [7]:
train_df = train_df.drop('id', axis=1)
train_df

,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
0,7.817315,576.179270,41.24308670850264,32.135501,4.0,0.803199,37.403527,1.963787,13.691147,62.494044,12.82491203459621,1018.8665053152533,A1,NaN,NaN,0.562096
1,24.785727,240.003973,1.3596482765960705,19.977460,8.0,0.479456,21.843315,0.241473,27.545096,43.851238,12.012043660984917,1025.6238537572883,D4,E00,dual-axis,0.396447
2,46.652695,687.612799,91.26536837560256,1.496401,4.0,0.822398,48.222882,4.191800,43.363708,NaN,1.814399755560454,1010.9226539809573,C3,E00,NaN,0.573776
3,53.339567,735.141179,96.19095521176159,18.491582,3.0,0.837529,46.295748,0.960567,57.720436,67.361473,8.736258932034128,1021.8466633134253,A1,NaN,dual-axis,0.629009
4,5.575374,12.241203,27.495073003585226,30.722697,6.0,0.551833,0.000000,0.898062,6.786263,3.632000,0.52268384077164,1008.5559577591927,B2,E00,fixed,0.341874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,16.868428,NaN,93.53031757838667,14.393967,3.0,0.738911,12.147711,3.005355,26.206810,1.733013,12.594122273332914,1018.3744670739436,B2,E02,tracking,0.664907
19996,53.415061,296.970303,93.98571391279083,25.997012,2.0,0.513061,0.000000,0.532119,65.000000,64.558667,0.9769909288128159,1016.081102065643,D4,E00,fixed,0.354070
19997,2.442727,660.328019,37.9689180401391,32.818396,9.0,0.548602,13.047950,4.075498,11.584869,57.730134,4.750937249871706,1009.6844614602336,D4,NaN,tracking,0.419734
19998,NaN,632.760700,43.01470184078199,19.063517,4.0,NaN,0.000000,1.068906,21.149351,78.123689,11.304158443374758,1006.6738746072241,A1,E00,tracking,0.661963


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   temperature         18999 non-null  float64
 1   irradiance          19013 non-null  float64
 2   humidity            20000 non-null  object 
 3   panel_age           18989 non-null  float64
 4   maintenance_count   18973 non-null  float64
 5   soiling_ratio       18990 non-null  float64
 6   voltage             19007 non-null  float64
 7   current             19023 non-null  float64
 8   module_temperature  19022 non-null  float64
 9   cloud_coverage      18990 non-null  float64
 10  wind_speed          20000 non-null  object 
 11  pressure            20000 non-null  object 
 12  string_id           20000 non-null  object 
 13  error_code          14088 non-null  object 
 14  installation_type   14972 non-null  object 
 15  efficiency          20000 non-null  float64
dtypes: f

In [9]:
train_df['temperature'].value_counts()

temperature
0.000000      389
60.000000      32
100.000000      2
25.311591       1
7.817315        1
             ... 
28.091359       1
37.652488       1
7.981089        1
21.097337       1
53.339567       1
Name: count, Length: 18579, dtype: int64

In [10]:
train_df['wind_speed'].value_counts()

wind_speed
badval                42
error                 41
unknown               36
1.4004639058482033     1
4.676583637669918      1
                      ..
9.196479438626795      1
0.7985304351521505     1
7.767053830591635      1
3.0572137101487558     1
8.321732514520054      1
Name: count, Length: 19884, dtype: int64

In [11]:
train_df['pressure'].value_counts()

pressure
unknown               46
error                 45
badval                44
1012.8003752419361     1
1001.3287852158609     1
                      ..
1021.4074573669117     1
1008.7504815771696     1
1014.7401210245107     1
1006.4182950874194     1
1030.4971803983756     1
Name: count, Length: 19868, dtype: int64

In [12]:
for i in ['humidity','wind_speed','pressure']:
    train_df[i] = pd.to_numeric(train_df[i].replace(['unknown', 'badval', 'error'], np.nan), errors='coerce').fillna(0)

In [13]:
train_df['installation_type'].value_counts()

installation_type
tracking     5067
fixed        4990
dual-axis    4915
Name: count, dtype: int64

In [14]:
train_df.isna().sum()

temperature           1001
irradiance             987
humidity                 0
panel_age             1011
maintenance_count     1027
soiling_ratio         1010
voltage                993
current                977
module_temperature     978
cloud_coverage        1010
wind_speed               0
pressure                 0
string_id                0
error_code            5912
installation_type     5028
efficiency               0
dtype: int64

In [15]:
train_df[['error_code', 'installation_type']] = train_df[['error_code', 'installation_type']].fillna('unknown')


In [16]:
train_df['installation_type'].value_counts()

installation_type
tracking     5067
unknown      5028
fixed        4990
dual-axis    4915
Name: count, dtype: int64

In [17]:
(train_df.isna().sum()/train_df.shape[0]) * 100 # percentage of nan values

temperature           5.005
irradiance            4.935
humidity              0.000
panel_age             5.055
maintenance_count     5.135
soiling_ratio         5.050
voltage               4.965
current               4.885
module_temperature    4.890
cloud_coverage        5.050
wind_speed            0.000
pressure              0.000
string_id             0.000
error_code            0.000
installation_type     0.000
efficiency            0.000
dtype: float64

In [18]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   temperature         18999 non-null  float64
 1   irradiance          19013 non-null  float64
 2   humidity            20000 non-null  float64
 3   panel_age           18989 non-null  float64
 4   maintenance_count   18973 non-null  float64
 5   soiling_ratio       18990 non-null  float64
 6   voltage             19007 non-null  float64
 7   current             19023 non-null  float64
 8   module_temperature  19022 non-null  float64
 9   cloud_coverage      18990 non-null  float64
 10  wind_speed          20000 non-null  float64
 11  pressure            20000 non-null  float64
 12  string_id           20000 non-null  object 
 13  error_code          20000 non-null  object 
 14  installation_type   20000 non-null  object 
 15  efficiency          20000 non-null  float64
dtypes: f

In [19]:
for col in ['maintenance_count', 'soiling_ratio', 'voltage', 'current', 'module_temperature', 'cloud_coverage']:
    train_df[col] = train_df[col].fillna(train_df[col].mean())

In [20]:
for col in ['temperature','irradiance','panel_age']:
    train_df[col] = train_df[col].fillna(train_df[col].mean())

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   temperature         20000 non-null  float64
 1   irradiance          20000 non-null  float64
 2   humidity            20000 non-null  float64
 3   panel_age           20000 non-null  float64
 4   maintenance_count   20000 non-null  float64
 5   soiling_ratio       20000 non-null  float64
 6   voltage             20000 non-null  float64
 7   current             20000 non-null  float64
 8   module_temperature  20000 non-null  float64
 9   cloud_coverage      20000 non-null  float64
 10  wind_speed          20000 non-null  float64
 11  pressure            20000 non-null  float64
 12  string_id           20000 non-null  object 
 13  error_code          20000 non-null  object 
 14  installation_type   20000 non-null  object 
 15  efficiency          20000 non-null  float64
dtypes: f

In [22]:

X, y = train_df.drop('efficiency', axis=1), train_df['efficiency']

In [23]:
numerical_features = X.select_dtypes(include=np.number).columns.tolist()

categorical_features = X.select_dtypes(include='object').columns.tolist()

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='drop'
)

In [24]:
preprocessor.fit(X)
processed_values = preprocessor.transform(X)

feature_names_out = preprocessor.get_feature_names_out()
X_train_processed = pd.DataFrame(processed_values, columns=feature_names_out)
print(f"Shape of processed data: {X_train_processed.shape}") # you can see that columns have increased that is because of oneHot encoding for the categorical columns

Shape of processed data: (20000, 24)


In [25]:
#model selection and comparison
def competition_score_metric(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return 100 * (1 - rmse)
competition_scorer_sklearn = make_scorer(competition_score_metric, greater_is_better=True)

In [26]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "Lasso": Lasso(alpha=0.01, random_state=42),
    "XGBoost": xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42, n_jobs=1, learning_rate=0.1, max_depth=5, verbosity=0),
    "LightGBM": lgb.LGBMRegressor(objective='regression_l1', n_estimators=100, random_state=42, n_jobs=1, learning_rate=0.1, max_depth=5, verbose=-1, verbosity=-1),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=1, max_depth=10, min_samples_leaf=5, verbose=0),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42, learning_rate=0.1, max_depth=5, verbose=0),
}

In [27]:
results = {}
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [28]:
for name, model_instance in models.items():
    print(f"Training {name}...")
    # cross_val_score clones the estimator internally.
    # Ensure n_jobs=1 for cross_val_score to avoid multiprocessing issues on some systems for this step.
    cv_rmse_scores = np.sqrt(-cross_val_score(model_instance, X_train_processed, y, cv=kf, scoring='neg_mean_squared_error', n_jobs=1))
    cv_mae_scores = -cross_val_score(model_instance, X_train_processed, y, cv=kf, scoring='neg_mean_absolute_error', n_jobs=1)
    cv_r2_scores = cross_val_score(model_instance, X_train_processed, y, cv=kf, scoring='r2', n_jobs=1)
    
    competition_scores_cv = [100 * (1 - rmse) for rmse in cv_rmse_scores]
    results[name] = {
        "Mean RMSE": np.mean(cv_rmse_scores),
        "Std RMSE": np.std(cv_rmse_scores),
        "Mean MAE": np.mean(cv_mae_scores),
        "Mean R2": np.mean(cv_r2_scores),
        "Mean Competition Score": np.mean(competition_scores_cv)
    }
    print(f"{name} - Mean Competition Score: {results[name]['Mean Competition Score']:.4f}, Mean RMSE: {results[name]['Mean RMSE']:.4f}")

Training Linear Regression...
Linear Regression - Mean Competition Score: 89.5383, Mean RMSE: 0.1046
Training Ridge...
Ridge - Mean Competition Score: 89.5383, Mean RMSE: 0.1046
Training Lasso...
Lasso - Mean Competition Score: 89.3256, Mean RMSE: 0.1067
Training XGBoost...
XGBoost - Mean Competition Score: 89.6035, Mean RMSE: 0.1040
Training LightGBM...
LightGBM - Mean Competition Score: 89.6226, Mean RMSE: 0.1038
Training Random Forest...
Random Forest - Mean Competition Score: 89.5999, Mean RMSE: 0.1040
Training Gradient Boosting...
Gradient Boosting - Mean Competition Score: 89.5748, Mean RMSE: 0.1043


In [29]:
results_df = pd.DataFrame(results).T.sort_values(by="Mean Competition Score", ascending=False)
print("\nCross-validation Results:")
print(results_df)


Cross-validation Results:
                   Mean RMSE  Std RMSE  Mean MAE   Mean R2  \
LightGBM            0.103774  0.003660  0.050343  0.453333   
XGBoost             0.103965  0.003459  0.054136  0.451334   
Random Forest       0.104001  0.003639  0.054671  0.450932   
Gradient Boosting   0.104252  0.003672  0.054082  0.448288   
Ridge               0.104617  0.003465  0.055241  0.444469   
Linear Regression   0.104617  0.003465  0.055241  0.444469   
Lasso               0.106744  0.003013  0.058525  0.421768   

                   Mean Competition Score  
LightGBM                        89.622551  
XGBoost                         89.603500  
Random Forest                   89.599919  
Gradient Boosting               89.574825  
Ridge                           89.538277  
Linear Regression               89.538276  
Lasso                           89.325609  


In [30]:
print("\nHyperparameter Optimization...")
best_model_name = results_df.index[0] if not results_df.empty else "LightGBM"
print(f"Optimizing: {best_model_name}")
best_model_tuned = None 


Hyperparameter Optimization...
Optimizing: LightGBM


In [31]:
if best_model_name == "LightGBM":
    estimator_for_tuning = lgb.LGBMRegressor(objective='regression_l1', random_state=42, verbose=-1, verbosity=-1, n_jobs=1)
    param_grid_lgbm = {
        'n_estimators': [200, 500, 1000, 1500, 2000],
        'learning_rate': [0.01, 0.02, 0.05, 0.08, 0.1],
        'num_leaves': [20, 31, 50, 70, 90],
        'max_depth': [-1, 7, 10, 15],
        'min_child_samples': [10, 20, 50, 70],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'reg_alpha': [0, 0.001, 0.01, 0.1, 0.5, 1],
        'reg_lambda': [0, 0.001, 0.01, 0.1, 0.5, 1],
        'boosting_type': ['gbdt', 'dart']
    }
    random_search = RandomizedSearchCV(
        estimator=estimator_for_tuning,
        param_distributions=param_grid_lgbm,
        n_iter=50,
        cv=kf,
        scoring=competition_scorer_sklearn,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    random_search.fit(X_train_processed, y)
    print(f"Best parameters for LightGBM: {random_search.best_params_}")
    print(f"Best Competition Score from RandomizedSearch for LightGBM: {random_search.best_score_:.4f}")
    best_model_tuned = random_search.best_estimator_

elif best_model_name == "XGBoost":
    estimator_for_tuning = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, verbosity=0, n_jobs=1)
    param_grid_xgb = {
        'n_estimators': [200, 500, 1000, 1500, 2000],
        'learning_rate': [0.01, 0.02, 0.05, 0.08, 0.1],
        'max_depth': [3, 5, 7, 9],
        'min_child_weight': [1, 3, 5, 7],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 0.1, 0.2, 0.5],
        'reg_alpha': [0, 0.001, 0.01, 0.1, 0.5, 1],
        'reg_lambda': [1, 0.5, 0.1, 0.01, 0]
    }
    random_search = RandomizedSearchCV(
        estimator=estimator_for_tuning,
        param_distributions=param_grid_xgb,
        n_iter=50,
        cv=kf,
        scoring=competition_scorer_sklearn,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    random_search.fit(X_train_processed, y)
    print(f"Best parameters for XGBoost: {random_search.best_params_}")
    print(f"Best Competition Score from RandomizedSearch for XGBoost: {random_search.best_score_:.4f}")
    best_model_tuned = random_search.best_estimator_

else:
    print(f"Hyperparameter tuning for {best_model_name} not as extensive. Training with initial params.")
    best_model_tuned = models[best_model_name] 
    if hasattr(best_model_tuned, 'n_jobs') and best_model_tuned.n_jobs != 1:
        best_model_tuned.set_params(n_jobs=1)
    best_model_tuned.fit(X_train_processed, y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters for LightGBM: {'subsample': 1.0, 'reg_lambda': 0, 'reg_alpha': 0.001, 'num_leaves': 31, 'n_estimators': 500, 'min_child_samples': 70, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.9, 'boosting_type': 'dart'}
Best Competition Score from RandomizedSearch for LightGBM: 89.7382


In [32]:
#Final Model Training & Prediction ---
print("\nMaking predictions on the test set...")
train_predictions = best_model_tuned.predict(X_train_processed)
train_predictions = np.clip(train_predictions, 0, 1)

#Submission File Generation
print("\nGenerating submission file...")
submission_df = pd.DataFrame({'id': train_df.index, 'efficiency': train_predictions})
submission_df.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created successfully.")
print(submission_df.head())


Making predictions on the test set...

Generating submission file...
Submission file 'submission.csv' created successfully.
   id  efficiency
0   0    0.541071
1   1    0.405325
2   2    0.597631
3   3    0.593072
4   4    0.344543
